## Overview

This notebook builds a long context benchmark, as discussed by Federico, Arjun,
Harm, and Leandro. Unlike typical Code LLM benchmarks, this is a test
generation benchmark: we prompt the model with the implementation of a Python
function (and its docstring), and ask for a test suite. The result is scored in
two steps: if any test in the test suite fails, the score is zero. Otherwise,
we the tests are scored based on their coverage of the funciton's
implementation. To make the problem harder, we add several other functions to
the prompt to serve as distractors. There are enough distractors to exercise
models with very long context lengths (up to 128K tokens). We use two datasets:
HumanEval and MultiPL-T. Both have several Python functions. The HumanEval
functions should be decontaminated before training: their docstrings should not
appear in the training data. The MultiPL-T functions are functions extracted
from the Stack v1.2. Thus they are very likely to appear in models' training
data, but they are merely distractors.

In [41]:
import datasets
import random
import os
from typing import List
import itertools

In [42]:
# In case you're in an environment where you really want this to be set.
print(os.getenv("HF_DATASETS_CACHE"))

random.seed(42)

# This is likely an overestimate. But, it should be close enough and we don't need
# to be exact.
CHARS_PER_TOKEN = 2.5

None


In [43]:
humaneval = datasets.load_dataset("openai_humaneval", split="test")
# The MultiPL-T dataset is currently private, but will be public soon.
multiplt = datasets.load_dataset("nuprl/stack-dedup-python-testgen-starcoder-filter-inferred-v2", split="train")
# mutant generated dataset
mutant_ds = datasets.load_dataset("nuprl/humaneval-py-mutants", split="train")

Found cached dataset openai_humaneval (/home/elleven/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)
Found cached dataset parquet (/home/elleven/.cache/huggingface/datasets/nuprl___parquet/nuprl--stack-dedup-python-testgen-starcoder-filter-inferred-v2-8a147987b4874669/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/elleven/.cache/huggingface/datasets/nuprl___parquet/nuprl--humaneval-py-mutants-b940dfa114eb395a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [44]:
def get_distractors(approximate_token_count: int) -> List[str]:
    result = []
    result_chars = 0
    target_chars = int(approximate_token_count * CHARS_PER_TOKEN)
    while result_chars < target_chars:
        fn = random.choice(multiplt)["content"]
        result.append(fn)
        result_chars += len(fn)
    return result

def find_mutant_with_idx(humaneval_problem_index: int):
    for ex in mutant_ds:
        if f"_{humaneval_problem_index}_" in ex["name"]:
            return ex
    return None

def build_prompt(
        approximate_token_count: int,
        humaneval_problem_index: int,
        insert_where: str):
    distractors = get_distractors(approximate_token_count)
    target_problem = humaneval[humaneval_problem_index]
    target_function = target_problem["prompt"] + target_problem["canonical_solution"]
    if insert_where == "first half":
        insert_index = random.randint(0, len(distractors) // 2)    
    elif insert_where == "second half":
        insert_index = random.randint(len(distractors) // 2, len(distractors))
    else:
        raise ValueError(f"Unknown insert_where: {insert_where}")
    distractors.insert(insert_index, target_function)
    mutants = find_mutant_with_idx(humaneval_problem_index)["mutants"]
    mutants = mutants if mutants is not None else []
    return { 
        "prompt": "\n\n".join(distractors),
        "target_function": target_function,
        "humaneval_task_id": target_problem["task_id"],
        "task_id": f"LongBench_{target_problem['task_id']}_{approximate_token_count}_{insert_where}",
        "approx_token_count": approximate_token_count,
        "mutants": mutants,
        "target_function_name": target_problem["entry_point"]
    }


## Example Prompts

Some examples of prompts that we can construct.

With 0 as the number of target tokens, we get no distractors.

In [45]:
print(build_prompt(0, 53, "first half")["prompt"])



def add(x: int, y: int):
    """Add two numbers x and y
    >>> add(2, 3)
    5
    >>> add(5, 7)
    12
    """
    return x + y



With 400 target tokens, we get 1-2 distractors and the `where` argument starts to
make sense.

In [46]:
print(build_prompt(400, 53, "first half")["prompt"])



def add(x: int, y: int):
    """Add two numbers x and y
    >>> add(2, 3)
    5
    >>> add(5, 7)
    12
    """
    return x + y


def to_list(tup: tuple):
    """Convert from tuple to packed list

    Allows us to call function with arguments in a loop

    Parameters
    ----------
    tup: tuple
        tuple of objects to convert to packed list

    Raises
    ------
    ValueError
        If passed uneven number of arguments without a list. Please wrap your args in a list.

    Examples
    --------
    >>> to_list(([x1, x2, x3], [y1, y2, y3]))
    [[x1, y1], [x2, y2], [x3, y3]]
    >>> to_list(([x1], [y1]))
    [[x1, y1]]
    >>> to_list(([x1, x2, x3], ))
    [[x1], [x2], [x3]]
    >>> to_list((x1, ))
    [[x1]]
    >>> to_list((x1, y1))
    [[x1, y1]]
    >>> to_list((x1, x2, x3, y1, y2, y3))
    [[x1, y1], [x2, y2], [x3, y3]]
    """
    n_tup = len(tup)
    if n_tup == 0:
        return []
    if not isinstance(tup[0], list):
        # the first element is data
        if n

In [47]:
print(build_prompt(400, 53, "second half")["prompt"])

def HEXtoVOLTS(ADChexStr):
    """ADC: 1000.0 volts full scale (D1B6).
    (for scaling ADC input)
    HEXtoVOLTS('D1B6') => 1000
    HEXtoVOLTS('BCD4') =>  900 """
    return int(int(ADChexStr,16) / 53.686 + 0.5)

def erathostenes_sieve(n: int) -> dict:
    """
    1.6
    Returns a dictionary containing prime numbers
    It is not recommended to use a list (1) because of
    reallocation and (2) because of index handling : the prime
    numbers start at 2 and the list, 0; and because some numbers
    are removed, the index changes too
    """
    if n <= 0: raise ValueError("Argument can't be <= 0")

    primes = {n:None for n in range(2, n + 1)}

    # We must convert it as a list, because the size changes
    for prime in list(primes.keys()):
        for number in list(primes.keys()):
            # It is a composit number if it is a multiple of the prime
            if prime != number and number%prime == 0:
                del primes[number]

    return primes

def kewley_agn_oi(lo

## The Benchmark

There are a number of trivial problems in HumanEval, such as #53 shown above.
We want a subset of problems that have a range of difficulties. The following
ten problems have varying difficulty in several programming languages
and were picked by Francesca Lucchetti for MultiPL-T.

- HumanEval_100_make_a_pile
- HumanEval_13_greatest_common_divisor
- HumanEval_152_compare
- HumanEval_157_right_angle_triangle
- HumanEval_27_flip_case
- HumanEval_40_triples_sum_to_zero
- HumanEval_55_fib
- HumanEval_66_digitSum
- HumanEval_72_will_it_fly
- HumanEval_74_total_match

In [48]:
APPROXIMATE_TOKEN_COUNTS = [0, 8_000, 64_000, 128_000]
HUMANEVAL_PROBLEM_INDICES = [100, 13, 152, 157, 27, 40, 55, 66, 72, 74]
INSERT_WHERES = [ "first half", "second half"]

benchmark = datasets.Dataset.from_list(
    [build_prompt(*x) for x in itertools.product(APPROXIMATE_TOKEN_COUNTS, HUMANEVAL_PROBLEM_INDICES, INSERT_WHERES)])
benchmark

Dataset({
    features: ['prompt', 'target_function', 'humaneval_task_id', 'task_id', 'approx_token_count', 'mutants', 'target_function_name'],
    num_rows: 80
})

In [49]:
benchmark.to_json("benchmark.jsonl", lines=True)

Creating json from Arrow format: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.82ba/s]


10971445

In [50]:
benchmark[0]

{'prompt': '\ndef make_a_pile(n):\n    """\n    Given a positive integer n, you have to make a pile of n levels of stones.\n    The first level has n stones.\n    The number of stones in the next level is:\n        - the next odd number if n is odd.\n        - the next even number if n is even.\n    Return the number of stones in each level in a list, where element at index\n    i represents the number of stones in the level (i+1).\n\n    Examples:\n    >>> make_a_pile(3)\n    [3, 5, 7]\n    """\n    return [n + 2*i for i in range(n)]\n',
 'target_function': '\ndef make_a_pile(n):\n    """\n    Given a positive integer n, you have to make a pile of n levels of stones.\n    The first level has n stones.\n    The number of stones in the next level is:\n        - the next odd number if n is odd.\n        - the next even number if n is even.\n    Return the number of stones in each level in a list, where element at index\n    i represents the number of stones in the level (i+1).\n\n    Exa